## Transform Customer Data
1. Remove records with NULL customer_id 
1. Remove exact duplicate records
1. Remove duplicate records based on created_timestamp
1. CAST the columns to the correct Data Type
1. Write transformed data to the Silver schema

### 1. Remove records with NULL customer_id

In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not null

### 2. Remove exact duplicate records

In [0]:
select distinct * from gizmobox.bronze.v_customers 
where customer_id is not null
order by customer_id;


In [0]:
create or replace temporary view v_customers_distinct
as select distinct * from gizmobox.bronze.v_customers 
where customer_id is not null


In [0]:
select customer_id, 
max(created_timestamp) as max_created_timestamp
from v_customers_distinct
group by customer_id

In [0]:
with cte_max as (
  select customer_id, 
  max(created_timestamp) as max_created_timestamp
  from v_customers_distinct
  group by customer_id
)
select * from v_customers_distinct t 
join cte_max c on t.customer_id = c.customer_id and t.created_timestamp = c.max_created_timestamp

### 4. CAST the column values to the correct data type

In [0]:
with cte_max as 
(
  select customer_id,
         max(created_timestamp) as max_created_timestamp
  from v_customers_distinct
  group by customer_id
)
select cast(t.created_timestamp as timestamp) as created_timestamp,
       t.customer_id,
       t.customer_name,
       cast(t.date_of_birth as date) as date_of_birth,
       t.email,
       cast(t.member_since as date) as member_since,
       t.telephone
from v_customers_distinct t
join cte_max c
  on t.customer_id = c.customer_id 
 and t.created_timestamp = c.max_created_timestamp;


### 5. Write Data to a Delta Table

In [0]:
create table gizmobox.silver.customers
as
with cte_max as 
(
  select customer_id,
         max(created_timestamp) as max_created_timestamp
  from v_customers_distinct
  group by customer_id
)
select cast(t.created_timestamp as timestamp) as created_timestamp,
       t.customer_id,
       t.customer_name,
       cast(t.date_of_birth as date) as date_of_birth,
       t.email,
       cast(t.member_since as date) as member_since,
       t.telephone
from v_customers_distinct t
join cte_max m 
  on t.customer_id = m.customer_id 
  and t.created_timestamp = m.max_created_timestamp;


In [0]:
select * from gizmobox.silver.customers

In [0]:
describe extended gizmobox.silver.customers;